# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6d18761fa0>
├── 'a' --> tensor([[-1.4797, -1.5619,  1.3876],
│                   [ 0.7906,  0.2686,  0.2323]])
└── 'x' --> <FastTreeValue 0x7f6d18761f40>
    └── 'c' --> tensor([[ 0.2748, -0.5459, -0.9655, -0.1151],
                        [-0.1689,  1.3469,  0.6671,  0.2490],
                        [ 0.0405,  0.4272,  1.7937, -0.3741]])

In [4]:
t.a

tensor([[-1.4797, -1.5619,  1.3876],
        [ 0.7906,  0.2686,  0.2323]])

In [5]:
%timeit t.a

57.5 ns ± 0.0112 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6d18761fa0>
├── 'a' --> tensor([[ 0.3058,  1.2949,  0.0886],
│                   [ 0.6104, -1.1264,  0.5006]])
└── 'x' --> <FastTreeValue 0x7f6d18761f40>
    └── 'c' --> tensor([[ 0.2748, -0.5459, -0.9655, -0.1151],
                        [-0.1689,  1.3469,  0.6671,  0.2490],
                        [ 0.0405,  0.4272,  1.7937, -0.3741]])

In [7]:
%timeit t.a = new_value

66.2 ns ± 0.0718 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.4797, -1.5619,  1.3876],
               [ 0.7906,  0.2686,  0.2323]]),
    x: Batch(
           c: tensor([[ 0.2748, -0.5459, -0.9655, -0.1151],
                      [-0.1689,  1.3469,  0.6671,  0.2490],
                      [ 0.0405,  0.4272,  1.7937, -0.3741]]),
       ),
)

In [10]:
b.a

tensor([[-1.4797, -1.5619,  1.3876],
        [ 0.7906,  0.2686,  0.2323]])

In [11]:
%timeit b.a

51.7 ns ± 0.144 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 2.0907,  0.4332,  0.4508],
               [-1.1511, -0.9908, -0.8310]]),
    x: Batch(
           c: tensor([[ 0.2748, -0.5459, -0.9655, -0.1151],
                      [-0.1689,  1.3469,  0.6671,  0.2490],
                      [ 0.0405,  0.4272,  1.7937, -0.3741]]),
       ),
)

In [13]:
%timeit b.a = new_value

498 ns ± 0.191 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

728 ns ± 0.569 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 26.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 203 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 360 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6c61a1d610>
├── 'a' --> tensor([[[-1.4797, -1.5619,  1.3876],
│                    [ 0.7906,  0.2686,  0.2323]],
│           
│                   [[-1.4797, -1.5619,  1.3876],
│                    [ 0.7906,  0.2686,  0.2323]],
│           
│                   [[-1.4797, -1.5619,  1.3876],
│                    [ 0.7906,  0.2686,  0.2323]],
│           
│                   [[-1.4797, -1.5619,  1.3876],
│                    [ 0.7906,  0.2686,  0.2323]],
│           
│                   [[-1.4797, -1.5619,  1.3876],
│                    [ 0.7906,  0.2686,  0.2323]],
│           
│                   [[-1.4797, -1.5619,  1.3876],
│                    [ 0.7906,  0.2686,  0.2323]],
│           
│                   [[-1.4797, -1.5619,  1.3876],
│                    [ 0.7906,  0.2686,  0.2323]],
│           
│                   [[-1.4797, -1.5619,  1.3876],
│                    [ 0.7906,  0.2686,  0.2323]]])
└── 'x' --> <FastTreeValue 0x7f6c61a1dd90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.9 µs ± 62.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6d187ce040>
├── 'a' --> tensor([[-1.4797, -1.5619,  1.3876],
│                   [ 0.7906,  0.2686,  0.2323],
│                   [-1.4797, -1.5619,  1.3876],
│                   [ 0.7906,  0.2686,  0.2323],
│                   [-1.4797, -1.5619,  1.3876],
│                   [ 0.7906,  0.2686,  0.2323],
│                   [-1.4797, -1.5619,  1.3876],
│                   [ 0.7906,  0.2686,  0.2323],
│                   [-1.4797, -1.5619,  1.3876],
│                   [ 0.7906,  0.2686,  0.2323],
│                   [-1.4797, -1.5619,  1.3876],
│                   [ 0.7906,  0.2686,  0.2323],
│                   [-1.4797, -1.5619,  1.3876],
│                   [ 0.7906,  0.2686,  0.2323],
│                   [-1.4797, -1.5619,  1.3876],
│                   [ 0.7906,  0.2686,  0.2323]])
└── 'x' --> <FastTreeValue 0x7f6c61a43f40>
    └── 'c' --> tensor([[ 0.2748, -0.5459, -0.9655, -0.1151],
                        [-0.1689,  1.3469,  0.6671,  0.2490],
                 

In [23]:
%timeit t_cat(trees)

27.3 µs ± 43.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.2 µs ± 61.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.4797, -1.5619,  1.3876],
                [ 0.7906,  0.2686,  0.2323]],
       
               [[-1.4797, -1.5619,  1.3876],
                [ 0.7906,  0.2686,  0.2323]],
       
               [[-1.4797, -1.5619,  1.3876],
                [ 0.7906,  0.2686,  0.2323]],
       
               [[-1.4797, -1.5619,  1.3876],
                [ 0.7906,  0.2686,  0.2323]],
       
               [[-1.4797, -1.5619,  1.3876],
                [ 0.7906,  0.2686,  0.2323]],
       
               [[-1.4797, -1.5619,  1.3876],
                [ 0.7906,  0.2686,  0.2323]],
       
               [[-1.4797, -1.5619,  1.3876],
                [ 0.7906,  0.2686,  0.2323]],
       
               [[-1.4797, -1.5619,  1.3876],
                [ 0.7906,  0.2686,  0.2323]]]),
    x: Batch(
           c: tensor([[[ 0.2748, -0.5459, -0.9655, -0.1151],
                       [-0.1689,  1.3469,  0.6671,  0.2490],
                       [ 0.0405,  0.4272,  1.7937, -0.3741]],
         

In [26]:
%timeit Batch.stack(batches)

68.4 µs ± 481 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.4797, -1.5619,  1.3876],
               [ 0.7906,  0.2686,  0.2323],
               [-1.4797, -1.5619,  1.3876],
               [ 0.7906,  0.2686,  0.2323],
               [-1.4797, -1.5619,  1.3876],
               [ 0.7906,  0.2686,  0.2323],
               [-1.4797, -1.5619,  1.3876],
               [ 0.7906,  0.2686,  0.2323],
               [-1.4797, -1.5619,  1.3876],
               [ 0.7906,  0.2686,  0.2323],
               [-1.4797, -1.5619,  1.3876],
               [ 0.7906,  0.2686,  0.2323],
               [-1.4797, -1.5619,  1.3876],
               [ 0.7906,  0.2686,  0.2323],
               [-1.4797, -1.5619,  1.3876],
               [ 0.7906,  0.2686,  0.2323]]),
    x: Batch(
           c: tensor([[ 0.2748, -0.5459, -0.9655, -0.1151],
                      [-0.1689,  1.3469,  0.6671,  0.2490],
                      [ 0.0405,  0.4272,  1.7937, -0.3741],
                      [ 0.2748, -0.5459, -0.9655, -0.1151],
                      [-0.1689,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 359 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

303 µs ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
